In [1]:
from SBM_SDE_tensor import *
from obs_and_flow import *
#import seaborn as sns
import torch
from torch import nn
import torch.distributions as D
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import math
from tqdm import tqdm
import random
from torch.autograd import Function
import argparse
import os
import sys
from pathlib import Path
import shutil
import pandas as pd

In [2]:
torch.manual_seed(0)
devi = torch.device("".join(["cuda:",f'{cuda_id}']) if torch.cuda.is_available() else "cpu")

cuda_id = 1
dt = .1 #SDE discretization timestep.
t = 200 #Simulation run for T hours.
n = int(t / dt) 
t_span = np.linspace(0, t, n + 1)
t_span_tensor = torch.reshape(torch.Tensor(t_span), [1, n + 1, 1]) #T_span needs to be converted to tensor object. Additionally, facilitates conversion of I_S and I_D to tensor objects.
pretrain_lr = 1e-3
elbo_lr = 1e-3
niter = 250
piter = 50
batch_size = 2 #Number of sets of observation outputs to sample per set of parameters.
state_dim_SCON = 3 #Not including CO2 in STATE_DIM, because CO2 is an observation.
state_dim_SAWB = 4 #Not including CO2 in STATE_DIM, because CO2 is an observation.
prior_scale_factor = 1 / 3 #Prior standard deviations set at 0.25 of prior means.
obs_error_scale_factor = 1 / 3

In [3]:
temp_ref = 283

#System parameters from deterministic CON model
u_M = 0.002
a_SD = 0.33
a_DS = 0.33
a_M = 0.33
a_MSC = 0.5
k_S_ref = 0.000025
k_D_ref = 0.005
k_M_ref = 0.0002
Ea_S = 75
Ea_D = 50
Ea_M = 50

#SCON diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1

SCON_C_prior_means = {'u_M': u_M, 'a_SD': a_SD, 'a_DS': a_DS, 'a_M': a_M, 'a_MSC': a_MSC, 'k_S_ref': k_S_ref, 'k_D_ref': k_D_ref, 'k_M_ref': k_M_ref, 'Ea_S': Ea_S, 'Ea_D': Ea_D, 'Ea_M': Ea_M, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC}
SCON_SS_prior_means = {'u_M': u_M, 'a_SD': a_SD, 'a_DS': a_DS, 'a_M': a_M, 'a_MSC': a_MSC, 'k_S_ref': k_S_ref, 'k_D_ref': k_D_ref, 'k_M_ref': k_M_ref, 'Ea_S': Ea_S, 'Ea_D': Ea_D, 'Ea_M': Ea_M, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC}

#System parameters from deterministic AWB model
u_Q_ref = 0.2
Q = 0.002
a_MSA = 0.5
K_D = 200
K_U = 1
V_D_ref = 0.4
V_U_ref = 0.02
Ea_V_D = 75
Ea_V_U = 50
r_M = 0.0004
r_E = 0.00001
r_L = 0.0005

#SAWB diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
c_EEC = 0.001
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1
s_EEC = 0.1

SAWB_C_prior_means = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_D': K_D, 'K_U': K_U, 'V_D_ref': V_D_ref, 'V_U_ref': V_U_ref, 'Ea_V_D': Ea_V_D, 'Ea_V_U': Ea_V_U, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC, 'c_EEC': c_EEC}
SAWB_SS_prior_means = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_D': K_D, 'K_U': K_U, 'V_D_ref': V_D_ref, 'V_U_ref': V_U_ref, 'Ea_V_D': Ea_V_D, 'Ea_V_U': Ea_V_U, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC, 's_EEC': s_EEC}

#System parameters from deterministic model
u_Q_ref = 0.2
Q = 0.002
a_MSA = 0.5
K_DE = 200
K_UE = 1
V_DE_ref = 0.4
V_UE_ref = 0.02
Ea_V_DE = 75
Ea_V_UE = 50
r_M = 0.0004
r_E = 0.00001
r_L = 0.0005

#Diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
c_EEC = 0.001
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1
s_EEC = 0.1

SAWB_ECA_C_prior_means = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_DE': K_DE, 'K_UE': K_UE, 'V_DE_ref': V_DE_ref, 'V_UE_ref': V_UE_ref, 'Ea_V_DE': Ea_V_DE, 'Ea_V_UE': Ea_V_UE, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC, 'c_EEC': c_EEC}
SAWB_ECA_SS_prior_means = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_DE': K_DE, 'K_UE': K_UE, 'V_DE_ref': V_DE_ref, 'V_UE_ref': V_UE_ref, 'Ea_V_DE': Ea_V_DE, 'Ea_V_UE': Ea_V_UE, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC, 's_EEC': s_EEC}

In [4]:
#Obtain SOC and DOC pool litter inputs for all SBMs.
i_s_tensor = 0.001 + 0.0005 * torch.sin((2 * np.pi / (24 * 365)) * t_span_tensor) #Exogenous SOC input function
i_d_tensor = 0.0001 + 0.00005 * torch.sin((2 * np.pi / (24 * 365)) * t_span_tensor) #Exogenous DOC input function

In [5]:
#Mean field VI code block.

#Define mean-field class: consumes parameter dictionary with values used as initial mean values.
class MeanField(nn.Module):
    def __init__(self, init_params, sdev_scale_factor):
        super().__init__()

        #Use param dict to intialise the means for the mean-field approximations.
        means = []
        keys = []
        for key, value in init_params.items():
            keys += [key]
            means += [value]
        self.means = nn.Parameter(torch.Tensor(means))
        self.sds = nn.Parameter(self.means * sdev_scale_factor)
        #Save keys for forward output.
        self.keys = keys

    def forward(self, n = 30):
        #Update posterior.
        q_dist = D.normal.Normal(LowerBound.apply(self.means, 1e-6), LowerBound.apply(self.sds, 1e-8))
        #q_dist = D.log_normal.LogNormal(LowerBound.apply(self.means, 1e-6), LowerBound.apply(self.sds, 1e-8)) #Testing LogNormal sampling object.
        #Sample theta ~ q(theta).
        samples = LowerBound.apply(q_dist.rsample([n]), 1e-6)
        #samples_log = LowerBound.apply(q_dist.rsample([n]), 1e-6)
        #samples = torch.exp(samples_log) #Exp transformation of log sample SDE theta values for SDE use.
        #Evaluate log prob of theta samples.
        log_q_theta = torch.sum(q_dist.log_prob(samples), -1) #Shape of n.
        #log_q_theta = torch.sum(q_dist.log_prob(samples_log), -1) #Shape of n.
        #Return samples in same dictionary format.
        dict_out = {} #Define dictionary with n samples for each parameter.
        for key, sample in zip(self.keys, torch.split(samples, 1, -1),):
            dict_out[f"{key}"] = sample.squeeze(1) #Each sample is of shape [n].
        #Return samples in dictionary and tensor format.
        return dict_out, samples, log_q_theta

In [6]:
def calc_log_lik(C_PATH, T_SPAN_TENSOR, DT, I_S_TENSOR, I_D_TENSOR, DRIFT_DIFFUSION, PARAMS_DICT, TEMP_GEN, TEMP_REF):
    drift, diffusion_sqrt = DRIFT_DIFFUSION(C_PATH[:, :-1, :], T_SPAN_TENSOR[:, :-1, :], I_S_TENSOR[:, :-1, :], I_D_TENSOR[:, :-1, :], PARAMS_DICT, TEMP_GEN, TEMP_REF)
    print('\nDrift = ', drift)
    print('\nDiffusion = ', diffusion_sqrt)
    euler_maruyama_state_sample_object = D.multivariate_normal.MultivariateNormal(loc = C_PATH[:, :-1, :] + drift * DT, scale_tril = diffusion_sqrt * math.sqrt(DT))
    return euler_maruyama_state_sample_object.log_prob(C_PATH[:, 1:, :]).sum(-1)

In [9]:
def train(DEVICE, PRETRAIN_LR, ELBO_LR, NITER, PRETRAIN_ITER, BATCH_SIZE,
          PRIOR_SCALE_FACTOR, SDEFLOW, ObsModel, csv_to_obs_df, DATA_CSV,
          OBS_ERROR_SCALE_FACTOR, STATE_DIM, T, DT, N, T_SPAN_TENSOR,
          I_S_TENSOR, I_D_TENSOR, DRIFT_DIFFUSION, PARAM_PRIOR_MEANS_DICT,
          TEMP_GEN, TEMP_REF, ANALYTICAL_STEADY_STATE_INIT,
          LR_DECAY = 0.1, DECAY_STEP_SIZE = 100000, LEARN_THETA = False, PRINT_EVERY = 10):
    if PRETRAIN_ITER >= NITER:
        raise Exception("PRETRAIN_ITER must be < NITER.")
    
    # Load data
    obs_times, obs_means, obs_error = csv_to_obs_df(DATA_CSV, STATE_DIM + 1, T, OBS_ERROR_SCALE_FACTOR) #Import data CSV and extract observation times, means, and desired observation error standard deviation based on `obs_error_scale_factor`. 
    
    # Initialize models
    obs_model = ObsModel(DEVICE, obs_times, DT, obs_means[:-1, :], obs_error[:, :-1]) #Hack for bypassing ObsModel and SDEFlow dimension mismatch issue.
    net = SDEFlow(DEVICE, obs_model, STATE_DIM, T, DT, N,
                  I_S_TENSOR, I_D_TENSOR, cond_inputs=3).to(DEVICE) #Instantiate flow.
    prior_means_tensor = torch.Tensor(list(PARAM_PRIOR_MEANS_DICT.values())) #Convert prior mean dictionary values to tensor.
    
    #Establish approximate priors.
    theta_dict = {k: torch.tensor(v).expand(BATCH_SIZE) for k, v in PARAM_PRIOR_MEANS_DICT.items()}
    
    if LEARN_THETA:
        priors = D.normal.Normal(prior_means_tensor, prior_means_tensor * PRIOR_SCALE_FACTOR)
        q_theta = MeanField(PARAM_PRIOR_MEANS_DICT, PRIOR_SCALE_FACTOR)
    else:
        q_theta = None
    
    #Initiate optimizers.
    pretrain_optimizer = optim.Adam(net.parameters(), lr = PRETRAIN_LR, eps = 1e-7)
    elbo_params = list(net.parameters()) + list(q_theta.parameters()) if LEARN_THETA \
        else net.parameters()
    ELBO_optimizer = optim.Adamax(elbo_params, lr = ELBO_LR)
    
    # Record loss throughout training
    best_loss_norm = 1e10
    best_loss_ELBO = 1e20
    norm_losses = [] #[best_loss_norm] * 10
    ELBO_losses = [] #[best_loss_ELBO] * 10
    
    #Joint optimization of SDE and hidden (NN) parameters loop.
    with tqdm(total = NITER, desc = f'\nTrain Diffusion', position = -1) as tq:
        for it in range(NITER):
            net.train()
            C_PATH, log_prob = net(BATCH_SIZE) #Obtain paths with solutions at times after t0.
            
            #Control flow for LEARN_THETA setting.
            if LEARN_THETA:
                theta_dict, theta, log_q_theta = q_theta(BATCH_SIZE)
                print('\ntheta_dict = ', theta_dict)
                log_p_theta = priors.log_prob(theta).sum(-1)
                log_p_y_0_giv_x_0_and_theta = 0 #Temporary. log_p_y_0_giv_x_0_and_theta = obs_model(C_0, theta_dict)/obs_model_CO2(C_0_with_CO2, theta_dict)
            else:
                log_q_theta = torch.tensor(0.)
                log_p_theta = torch.tensor(0.)
                log_p_y_0_giv_x_0_and_theta = 0
            
            #Initial C conditions estimation based on theta.
            C_0 = LowerBound.apply(ANALYTICAL_STEADY_STATE_INIT(I_S_TENSOR[0, 0, 0].item(), I_D_TENSOR[0, 0, 0].item(), theta_dict), 1e-7) #Calculate deterministic initial conditions.
            print('\nC_0 =', C_0)
            #C0 = C0[(None,) * 2].repeat(BATCH_SIZE, 1, 1).to(DEVICE) #Commenting out because analytical steady state init functions now output tensors with appropriate batch size if argument into MeanField forward function is BATCH_SIZE. #Assign initial conditions to C_PATH.
            C_PATH = torch.cat([C_0.unsqueeze(1), C_PATH], 1) #Append deterministic CON initial conditions conditional on parameter values to C path. 
            print('\nC_PATH =', C_PATH)
            print('\nC_PATH mean =', C_PATH.mean(-2))
            
            if it <= PRETRAIN_ITER:
                pretrain_optimizer.zero_grad()
                #l1_norm_element = C_PATH - torch.mean(obs_model.mu, -1)
                #l1_norm = torch.sum(torch.abs(l1_norm_element)).mean()
                #best_loss_norm = l1_norm if l1_norm < best_loss_norm else best_loss_norm
                #norm_losses.append(l1_norm.item())
                l2_norm_element = C_PATH - torch.mean(obs_model.mu, -1)
                l2_norm = torch.sqrt(torch.sum(torch.square(l2_norm_element))).mean()
                best_loss_norm = l2_norm if l2_norm < best_loss_norm else best_loss_norm
                norm_losses.append(l2_norm.item())
                
                if it % PRINT_EVERY == 0:
                    ma_norm_loss = sum(norm_losses[-10:]) / len(norm_losses[-10:])
                    print(f"\nMoving average norm loss at {iter} iterations is: {ma_norm_loss}. Best norm loss value is: {best_loss_norm}.")
                    print('\nC_PATH mean =', C_PATH.mean(-2))
                    print('\nC_PATH =', C_PATH)
                #l1_norm.backward()
                l2_norm.backward()
                pretrain_optimizer.step()
            else:
                ELBO_optimizer.zero_grad()                
                log_lik = calc_log_lik(C_PATH, T_SPAN_TENSOR.to(DEVICE), dt,
                                       I_S_TENSOR.to(DEVICE), I_D_TENSOR.to(DEVICE),
                                       DRIFT_DIFFUSION, theta_dict, TEMP_GEN, TEMP_REF)
                neg_ELBO = -log_p_theta.mean() + log_q_theta.mean() - log_p_y_0_giv_x_0_and_theta\
                    - log_lik.mean() - obs_model(C_PATH, theta_dict) + log_prob.mean() #From equation 14 of Ryder et al., 2019.
                #neg_ELBO = -log_lik.mean() - obs_model(C_PATH) + log_prob.mean() #Old ELBO computation without joint density optimization.
                print('\nneg_ELBO_mean = ', neg_ELBO)
                best_loss_ELBO = neg_ELBO if neg_ELBO < best_loss_ELBO else best_loss_ELBO
                ELBO_losses.append(neg_ELBO)
                
                if it % PRINT_EVERY == 0:
                    ma_elbo_loss = sum(ELBO_losses[-10:]) / len(ELBO_losses[-10:])
                    print(f"\nMoving average ELBO loss at {iter} iterations is: {ma_elbo_loss}. Best ELBO loss value is: {best_loss_ELBO}.")
                neg_ELBO.backward()
                ELBO_optimizer.step()
            torch.nn.utils.clip_grad_norm_(net.parameters(), 3.0)
            if it % DECAY_STEP_SIZE == 0 and it > 0:
                ELBO_optimizer.param_groups[0]['lr'] *= LR_DECAY
            tq.update()
            
    return net, q_theta, ELBO_losses, norm_losses

In [10]:
net1, q_theta1, ELBO_losses1, norm_losses1 = train(devi, pretrain_lr, elbo_lr, niter, piter, batch_size, prior_scale_factor, SDEFlow, ObsModel, csv_to_obs_df, 'CON_synthetic_sol_df.csv', 0.1, state_dim_SCON, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SCON_C, SCON_C_prior_means, temp_gen, temp_ref, analytical_steady_state_init_CON)



Train Diffusion:   0%|          | 0/250 [00:00<?, ?it/s]


C_0 = tensor([[45.6603,  0.0715,  0.7147],
        [45.6603,  0.0715,  0.7147]])

C_PATH = tensor([[[45.6603,  0.0715,  0.7147],
         [ 0.8933,  0.3039,  0.7411],
         [ 1.4497,  0.8368,  0.6357],
         ...,
         [ 1.9114,  0.6088,  1.0139],
         [ 0.9866,  0.7673,  0.7185],
         [ 0.3679,  0.7938,  1.6856]],

        [[45.6603,  0.0715,  0.7147],
         [ 0.8933,  0.5536,  0.7700],
         [ 1.4497,  0.4485,  0.7829],
         ...,
         [ 0.7194,  0.7654,  0.4409],
         [ 0.3386,  0.8224,  0.9120],
         [ 0.7667,  1.0320,  1.2059]]], grad_fn=<CatBackward>)

C_PATH mean = tensor([[1.0396, 0.7963, 0.8179],
        [1.0554, 0.7931, 0.8037]], grad_fn=<MeanBackward1>)

Moving average norm loss at <built-in function iter> iterations is: 2820.906494140625. Best norm loss value is: 2820.906494140625.

C_PATH mean = tensor([[1.0396, 0.7963, 0.8179],
        [1.0554, 0.7931, 0.8037]], grad_fn=<MeanBackward1>)

C_PATH = tensor([[[45.6603,  0.0715,  0.7147],



Train Diffusion:   0%|          | 1/250 [00:02<08:22,  2.02s/it]


C_0 = tensor([[45.6603,  0.0715,  0.7147],
        [45.6603,  0.0715,  0.7147]])

C_PATH = tensor([[[45.6603,  0.0715,  0.7147],
         [ 0.8007,  2.5210,  0.6712],
         [ 0.7255,  0.8454,  0.5533],
         ...,
         [ 1.4717,  0.6152,  0.4859],
         [ 0.9379,  0.6799,  0.4529],
         [ 0.7644,  0.6169,  0.5172]],

        [[45.6603,  0.0715,  0.7147],
         [ 0.9292,  0.8634,  0.5240],
         [ 2.8498,  0.2098,  0.6013],
         ...,
         [ 0.5534,  0.6052,  0.6204],
         [ 1.1764,  0.5270,  0.4390],
         [ 1.3250,  0.5058,  0.6798]]], grad_fn=<CatBackward>)

C_PATH mean = tensor([[1.8342, 0.7364, 0.5815],
        [1.7128, 0.7392, 0.5803]], grad_fn=<MeanBackward1>)




Train Diffusion:   1%|          | 2/250 [00:03<07:51,  1.90s/it]

...


C_0 = tensor([[45.6603,  0.0715,  0.7147],
        [45.6603,  0.0715,  0.7147]])

C_PATH = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01],
         [2.5129e+00, 1.4875e+01, 2.9538e-02],
         [1.6927e+00, 2.1524e-01, 3.1396e-02],
         ...,
         [1.9132e+01, 2.7407e-02, 8.8417e-03],
         [2.7575e+00, 2.6143e-02, 5.6122e-03],
         [4.1536e-02, 2.6199e-02, 1.1440e-03]],

        [[4.5660e+01, 7.1469e-02, 7.1469e-01],
         [4.7251e-01, 9.0114e-01, 2.7242e-02],
         [1.0736e+01, 1.3745e+01, 2.7960e-02],
         ...,
         [6.5159e-02, 2.7718e-02, 1.0550e-02],
         [1.4957e+01, 3.0697e-02, 5.4549e-03],
         [1.4491e+01, 3.0868e-02, 1.1587e-03]]], grad_fn=<CatBackward>)

C_PATH mean = tensor([[1.8773e+01, 2.3957e-01, 1.8531e-02],
        [1.8852e+01, 2.0163e-01, 1.8450e-02]], grad_fn=<MeanBackward1>)




Train Diffusion:  18%|█▊        | 46/250 [01:17<05:38,  1.66s/it]


C_0 = tensor([[45.6603,  0.0715,  0.7147],
        [45.6603,  0.0715,  0.7147]])

C_PATH = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01],
         [1.9711e+00, 7.4327e+00, 3.0703e-02],
         [9.5895e+00, 2.2640e-01, 2.6940e-02],
         ...,
         [1.0993e+01, 2.9436e-02, 1.2953e-02],
         [7.9606e+00, 2.6587e-02, 7.0393e-03],
         [7.6073e+00, 3.1361e-02, 1.1741e-03]],

        [[4.5660e+01, 7.1469e-02, 7.1469e-01],
         [5.0412e-01, 7.4692e-01, 2.6545e-02],
         [1.4428e+00, 7.0368e+00, 3.0100e-02],
         ...,
         [1.1057e-01, 3.0280e-02, 1.2305e-02],
         [1.8769e+00, 3.0473e-02, 7.2534e-03],
         [2.1988e-01, 2.5894e-02, 1.1397e-03]]], grad_fn=<CatBackward>)

C_PATH mean = tensor([[19.0470,  0.0955,  0.0209],
        [19.3665,  0.0952,  0.0209]], grad_fn=<MeanBackward1>)




Train Diffusion:  19%|█▉        | 47/250 [01:19<05:44,  1.69s/it]


KeyboardInterrupt: 

In [8]:
net2, q_theta2, ELBO_losses2, norm_losses2 = train(devi, pretrain_lr, elbo_lr, niter, piter, batch_size, prior_scale_factor, SDEFlow, ObsModel, csv_to_obs_df, 'CON_synthetic_sol_df.csv', 0.1, state_dim_SCON, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SCON_C, SCON_C_prior_means, temp_gen, temp_ref, analytical_steady_state_init_CON, LEARN_THETA = True)



Train Diffusion:   0%|          | 0/250 [00:00<?, ?it/s]


theta_dict =  {'u_M': tensor([0.0026, 0.0019], grad_fn=<SqueezeBackward1>), 'a_SD': tensor([0.4923, 0.5238], grad_fn=<SqueezeBackward1>), 'a_DS': tensor([0.4049, 0.3852], grad_fn=<SqueezeBackward1>), 'a_M': tensor([0.2982, 0.2780], grad_fn=<SqueezeBackward1>), 'a_MSC': tensor([0.6130, 0.2935], grad_fn=<SqueezeBackward1>), 'k_S_ref': tensor([1.7411e-05, 2.8036e-05], grad_fn=<SqueezeBackward1>), 'k_D_ref': tensor([0.0037, 0.0054], grad_fn=<SqueezeBackward1>), 'k_M_ref': tensor([0.0001, 0.0002], grad_fn=<SqueezeBackward1>), 'Ea_S': tensor([101.9323,  96.6047], grad_fn=<SqueezeBackward1>), 'Ea_D': tensor([55.7031, 40.1225], grad_fn=<SqueezeBackward1>), 'Ea_M': tensor([66.2807, 52.1877], grad_fn=<SqueezeBackward1>), 'c_SOC': tensor([1.0407, 1.0143], grad_fn=<SqueezeBackward1>), 'c_DOC': tensor([0.0153, 0.0120], grad_fn=<SqueezeBackward1>), 'c_MBC': tensor([0.1339, 0.0678], grad_fn=<SqueezeBackward1>)}

C_0 = tensor([[70.7924,  0.1176,  2.1779],
        [44.3364,  0.1088,  1.1591]], grad_fn



Train Diffusion:   0%|          | 1/250 [00:01<07:33,  1.82s/it]


theta_dict =  {'u_M': tensor([0.0005, 0.0020], grad_fn=<SqueezeBackward1>), 'a_SD': tensor([0.3980, 0.3361], grad_fn=<SqueezeBackward1>), 'a_DS': tensor([0.2354, 0.4278], grad_fn=<SqueezeBackward1>), 'a_M': tensor([0.1408, 0.3372], grad_fn=<SqueezeBackward1>), 'a_MSC': tensor([0.7173, 0.4950], grad_fn=<SqueezeBackward1>), 'k_S_ref': tensor([2.5771e-05, 2.2529e-05], grad_fn=<SqueezeBackward1>), 'k_D_ref': tensor([0.0055, 0.0044], grad_fn=<SqueezeBackward1>), 'k_M_ref': tensor([0.0002, 0.0002], grad_fn=<SqueezeBackward1>), 'Ea_S': tensor([77.3056, 87.3640], grad_fn=<SqueezeBackward1>), 'Ea_D': tensor([59.6544, 59.5062], grad_fn=<SqueezeBackward1>), 'Ea_M': tensor([41.5930, 41.4071], grad_fn=<SqueezeBackward1>), 'c_SOC': tensor([1.3883, 1.5022], grad_fn=<SqueezeBackward1>), 'c_DOC': tensor([0.0181, 0.0091], grad_fn=<SqueezeBackward1>), 'c_MBC': tensor([0.1718, 0.1150], grad_fn=<SqueezeBackward1>)}

C_0 = tensor([[43.5752,  0.0914,  0.2026],
        [52.4632,  0.0821,  0.6788]], grad_fn=<



Train Diffusion:   1%|          | 2/250 [00:03<07:15,  1.76s/it]

...


theta_dict =  {'u_M': tensor([0.0029, 0.0041], grad_fn=<SqueezeBackward1>), 'a_SD': tensor([0.1878, 0.2762], grad_fn=<SqueezeBackward1>), 'a_DS': tensor([0.4068, 0.3701], grad_fn=<SqueezeBackward1>), 'a_M': tensor([0.3730, 0.5142], grad_fn=<SqueezeBackward1>), 'a_MSC': tensor([0.4602, 0.7497], grad_fn=<SqueezeBackward1>), 'k_S_ref': tensor([0.0019, 0.0019], grad_fn=<SqueezeBackward1>), 'k_D_ref': tensor([0.0073, 0.0108], grad_fn=<SqueezeBackward1>), 'k_M_ref': tensor([0.0034, 0.0034], grad_fn=<SqueezeBackward1>), 'Ea_S': tensor([78.7069, 84.1664], grad_fn=<SqueezeBackward1>), 'Ea_D': tensor([41.3279, 55.0926], grad_fn=<SqueezeBackward1>), 'Ea_M': tensor([58.2869, 76.2135], grad_fn=<SqueezeBackward1>), 'c_SOC': tensor([1.5739, 0.9366], grad_fn=<SqueezeBackward1>), 'c_DOC': tensor([0.0224, 0.0216], grad_fn=<SqueezeBackward1>), 'c_MBC': tensor([0.1265, 0.1362], grad_fn=<SqueezeBackward1>)}

C_0 = tensor([[0.5730, 0.0322, 0.0269],
        [0.6045, 0.0294, 0.0350]], grad_fn=<LowerBoundBack



Train Diffusion:  44%|████▍     | 111/250 [03:00<04:14,  1.83s/it]


theta_dict =  {'u_M': tensor([0.0001, 0.0022], grad_fn=<SqueezeBackward1>), 'a_SD': tensor([0.5056, 0.2628], grad_fn=<SqueezeBackward1>), 'a_DS': tensor([0.4679, 0.0369], grad_fn=<SqueezeBackward1>), 'a_M': tensor([0.3577, 0.3994], grad_fn=<SqueezeBackward1>), 'a_MSC': tensor([0.6221, 0.7479], grad_fn=<SqueezeBackward1>), 'k_S_ref': tensor([0.0019, 0.0015], grad_fn=<SqueezeBackward1>), 'k_D_ref': tensor([0.0064, 0.0131], grad_fn=<SqueezeBackward1>), 'k_M_ref': tensor([0.0034, 0.0034], grad_fn=<SqueezeBackward1>), 'Ea_S': tensor([67.5512, 65.5300], grad_fn=<SqueezeBackward1>), 'Ea_D': tensor([45.9783, 47.9955], grad_fn=<SqueezeBackward1>), 'Ea_M': tensor([41.3075, 61.5144], grad_fn=<SqueezeBackward1>), 'c_SOC': tensor([0.6329, 0.6910], grad_fn=<SqueezeBackward1>), 'c_DOC': tensor([0.0217, 0.0215], grad_fn=<SqueezeBackward1>), 'c_MBC': tensor([0.0680, 0.0674], grad_fn=<SqueezeBackward1>)}

C_0 = tensor([[0.7088, 0.1224, 0.0043],
        [0.6698, 0.0245, 0.0153]], grad_fn=<LowerBoundBack


Train Diffusion:  44%|████▍     | 111/250 [03:02<03:48,  1.64s/it]


KeyboardInterrupt: 